## Overview

Point-of-Sale Streaming with Delta Live Tables (DLT) là một giải pháp ứng dụng Delta Live Tables để xây dựng kiến trúc lakehouse gần real-time nhằm tính toán lượng hàng tồn kho hiện tại cho nhiều sản phẩm tại các địa điểm cửa hàng khác nhau. Thay vì chuyển trực tiếp từ dữ liệu raw đã được ingest sang lượng tồn kho đã tính toán, giải pháp này chia logic thành hai giai đoạn riêng biệt.</p>

<img src='./flow-of-project.png' width=800>

### Ở phần Databrick
Trong giai đoạn đầu (Bronze-to-Silver ETL như được minh họa trong hình), dữ liệu đã ingest được transform để dễ truy cập hơn. Các hành động được thực hiện trên dữ liệu ở giai đoạn này, chẳng hạn như decomposing các nested arrays, deduplication các bản ghi, v.v.,. Các bảng được ghi vào ở giai đoạn này đại diện cho layer Silver của kiến trúc lakehouse [medallion architecture](https://databricks.com/glossary/medallion-architecture).

Trong giai đoạn tiếp theo (Silver-to-Gold ETL như được minh họa trong hình), các bảng Silver được sử dụng để tạo ra các đầu ra phù hợp với yêu cầu kinh doanh, cụ thể là tính toán lượng tồn kho hiện tại. Dữ liệu này được ghi vào bảng đại diện cho layer Gold của kiến trúc.

### Phần Fabric
Dữ liệu được ghi nhận trong IOT Hub, và cũng được stream vào Microsoft Fabric để phục vụ cho việc Visualize và phân tích.

Eventstream: Nhận dữ liệu real-time từ IotHub và chuyển đổi để sử dụng trong các ứng dụng downstream.

Eventhouse: Lưu trữ dữ liệu đã xử lý, tạo nền tảng để xây dựng báo cáo và phân tích chuyên sâu.

Real-Time Dashboard: Sử dụng dữ liệu đã xử lý để hiển thị thông tin tồn kho một cách trực quan, giúp các nhà quản lý theo dõi và tối ưu hóa hoạt động kinh doanh.



## The Notebooks

Excluding this one, the solution accelerator is comprised of 5 notebooks. These notebooks are:</p>

* *POS 01: Environment Setup*
* *POS 02: Data Generation*
* *POS 03: Bronze-to-Silver ETL*
* *POS 04: Silver-to-Gold ETL*
* *POS 05: DLT Job Scheduling*

The *POS 01: Environment Setup* notebook này cung cấp hướng dẫn thiết lập môi trường để ingest dữ liệu. Việc thiết lập bao gồm cấu hình các tài nguyên đám mây bên ngoài workspace của Databricks. Mặc dù tất cả các nhà cung cấp dịch vụ đám mây trên Databricks đều cung cấp cơ chế lưu trữ và ingest dữ liệu, giải pháp này sử dụng các dịch vụ của Azure để minh họa. 

The *POS 02: Data Generation* Notebook này đọc dữ liệu từ mô phỏng 30 ngày của dữ liệu POS từ hai cửa hàng giả lập, sau đó tạo ra một luồng stream dữ liệu nhắm đến môi trường IotHub được thiết lập trong *POS 01*. Dữ liệu mô phỏng được phát lại với tốc độ gấp 10 lần để cân bằng giữa việc kiểm tra quá trình streaming trong khoảng thời gian dài và thời gian thực thi ngắn hơn (không cần chạy trong suốt 30 ngày). Logic trong notebook này chỉ phụ thuộc hạn chế vào môi trường Databricks. Code được viết trong notebook Databricks thay vì một script Python độc lập nhằm mục đích dễ dàng thích nghi với chức nagnw Delta Live Table.

The *POS 03: Bronze-to-Silver ETL* Đây là bước đầu tiên trong quy trình xử lý dữ liệu. Tại bước này:

    - Dữ liệu từ các giao dịch POS và sự kiện tồn kho được lấy và xử lý gần như real-time.
    - Ngoài ra, 3 bảng dữ liệu tĩnh (ít thay đổi) cũng được tạo ra.
    - Điểm nổi bật: Bước này có thể xử lý và hỗ trợ nhiều loại bảng dữ liệu khác nhau, kể cả những bảng có tần suất cập nhật không đồng đều.

The *POS 04: Silver-to-Gold ETL* Đây là bước thứ hai, nơi dữ liệu từ bảng Silver được dùng để tạo ra kết quả kinh doanh cuối cùng. Cụ thể:

    - Tính toán lượng tồn kho hiện tại của các cửa hàng dựa trên dữ liệu từ bước trước.
    - Vì dữ liệu ở bảng Silver được lưu trữ trong định dạng Delta Lake, mọi thay đổi từ bước trước sẽ được cập nhật ngay lập tức trong bước này, đảm bảo luồng dữ liệu liên tục và nhanh chóng.

The *POS 05: DLT Job Scheduling* Bước này dùng để lên lịch chạy tự động cho các quy trình xử lý ở POS 03 và POS 04.

    - Các bước xử lý trước đó không cần chạy thủ công mà sẽ được công cụ DLT (Delta Live Tables) tự động hóa hoàn toàn.
    - Nhờ vậy, quy trình trở nên đơn giản, mượt mà và tiết kiệm thời gian.

## Structured Streaming vs. Delta Live Tables

These notebooks represent a revision of a solution accelerator published in 2021.  In [that accelerator](https://databricks.com/blog/2021/09/09/real-time-point-of-sale-analytics-with-a-data-lakehouse.html), the overall workflow is implemented using [Spark Structured Streaming](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) in a manner consistent with what is described above.

**NOTE** In the original solution accelerator on which this version is based, *POS 03* and *POS 04* had slightly different names.  The purpose of these notebooks between the two accelerators is the same regardless of the naming changes.

By introducing Delta Live Tables (DLT), we don't radically change the implementation of the streaming workflows. Instead, the use of DLT provides a wrapper on our workflows that enables orchestration, monitoring and many other enhancements we'd otherwise need to implement ourselves. In this regard, DLT complements Spark Structured Streaming and doesn't replace it.  If you compare the relevant notebooks between this and the previous solution accelerator, you'll be able to see how these two technologies work with each other.

That said, DLT does provide some features which can simplify the implementation of our logic.  In the original accelerator notebooks, we spent quite a bit of time describing a work-around that would enable us to perform a complex join between static and streaming objects.  DLT simplifies that aspect of this workflow and allows us to remove a bit of code from notebooks *POS 03* and *POS 04*.

The simplification of the logic is a welcome feature of DLT.  Still, DLT, as a new technology, does impose some constraints that have required us to revise some aspects of our code.  First, DLT notebooks must be self-contained, *i.e.* they cannot call other notebooks.  This has required us to copy some configuration logic from *POS 01* into *POS 03*.  Second, DLT does not yet support Azure IOT Hub Event Hub endpoints.  This has required us to reconfigure the streaming dataset reading data from the IOT Hub to leverage that service's Kafka endpoint.  Finally, notebooks defining DLT workflows are not run interactively and instead must be scheduled.  We've introduced an additional notebook, *POS 05*, to address the scheduling aspects for notebooks *POS 03* and *POS 04*.


&copy; 2022 Databricks, Inc. All rights reserved. The source in this notebook is provided subject to the [Databricks License](https://databricks.com/db-license-source).  All included or referenced third party libraries are subject to the licenses set forth below.

| library                                | description             | license    | source                                              |
|----------------------------------------|-------------------------|------------|-----------------------------------------------------|
| azure-iot-device                                     | Microsoft Azure IoT Device Library | MIT    | https://pypi.org/project/azure-iot-device/                       |
| azure-storage-blob                                | Microsoft Azure Blob Storage Client Library for Python| MIT        | https://pypi.org/project/azure-storage-blob/      |